1. Neural Network Cost Function

In [238]:
import numpy as np
import scipy.io
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.misc
from scipy.special import expit
import random

In [239]:
random.seed(13)

In [240]:
data = scipy.io.loadmat('ex4data1.mat')

In [241]:
X = data['X']
X = np.column_stack((np.ones(X.shape[0]), X))
y = data['y']
m = X.shape[0]
K = 10
y_true = np.zeros((m, K))
for i in range(m):
    y_true[i, y[i] - 1] = 1

In [242]:
weights = scipy.io.loadmat('ex4weights.mat')

In [243]:
Theta1 = weights['Theta1']
Theta2 = weights['Theta2']

In [244]:
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'Theta1 shape: {Theta1.shape}')
print(f'Theta2 shape: {Theta2.shape}')

X shape: (5000, 401)
y shape: (5000, 1)
Theta1 shape: (25, 401)
Theta2 shape: (10, 26)


In [247]:
def nnCostFunction(X, y, Theta1, Theta2, lam = 0):
    m = X.shape[0]
    K = 10
    a2 = expit(X @ Theta1.T)
    a2 = np.column_stack((np.ones(m), a2)) # a2 shape : (5000, 26), Hidden Layer
    a3 = expit(a2 @ Theta2.T) # a3 shape: (5000, 10), Output Layer
    cost = 0
    '''
    for i in range(m):
        for k in range(K):
            cost += -y_true[i, k] * np.log(a3[i,k]) - (1 - y_true[i,k]) * np.log(1 - a3[i,k])
    cost = cost/m
    '''
    yy = y_true.flatten()
    a3 = a3.flatten()
    term1 = -yy.T*np.log(a3)
    term2 = (1-yy.T)*np.log(1-a3)
    cost = np.sum((term1 - term2))/m
    reg = lam/(2*m)*(np.sum(Theta1 * Theta1)+ np.sum(Theta2 * Theta2))
    return cost + reg


In [249]:
cost = nnCostFunction(X, y, Theta1, Theta2)
print(cost)

0.2876291651613189


In [250]:
cost = nnCostFunction(X, y, Theta1, Theta2, lam = 1)
print(cost)

0.384487796242894


2 Backpropagation

In [251]:
def SigmoidGradient(z):
    g = expit(z)
    gradient = g * (1- g)
    return gradient

In [252]:
# test sigmoidGradient with matrix, vector
s = SigmoidGradient(np.zeros((3, 2)))
print(s)
s = SigmoidGradient(np.zeros(3))
print(s)

[[0.25 0.25]
 [0.25 0.25]
 [0.25 0.25]]
[0.25 0.25 0.25]


In [253]:
def randInitializeWeights(X, y, n_t1):
    '''
    y should be in shape of (5000, 10)
    '''
    np.random.seed(13)
    n_in = X.shape[1]
    n_out = y.shape[1]
    eplison_ini = 0.12
    t1_ini = np.random.rand(n_t1, n_in) * 2 * eplison_ini - eplison_ini
    t2_ini = np.random.rand(n_out, n_t1 + 1) * 2 * eplison_ini - eplison_ini
    
    return t1_ini, t2_ini, eplison_ini

t1_ini, t2_ini, eplison_ini = randInitializeWeights(X, y_true, 25)
print(t1_ini.shape, t2_ini.shape)


(25, 401) (10, 26)


In [254]:
t2_ini

array([[ 0.0361128 ,  0.00163764,  0.10451519, -0.00730572,  0.05729893,
         0.11286916,  0.08148461,  0.04677388,  0.10150134, -0.07028835,
        -0.06619306,  0.01892042,  0.00305671, -0.08378777,  0.09818563,
        -0.04062996, -0.01563425,  0.11000729, -0.02168252, -0.09855389,
         0.11695874,  0.10743737,  0.07068408, -0.11393802, -0.01878151,
         0.03167481],
       [ 0.10050056,  0.03264228, -0.08201097,  0.10373032, -0.02062659,
         0.03903743,  0.05827185, -0.04175627,  0.10103395,  0.10352145,
         0.05585771,  0.04016006,  0.10194907,  0.07458021,  0.07004874,
        -0.09252782, -0.00386945, -0.0815099 , -0.10220025,  0.11991605,
        -0.01695089,  0.10339707, -0.11323199, -0.05244234,  0.02095583,
        -0.08166763],
       [ 0.03324483, -0.0154486 ,  0.1105214 , -0.11963491, -0.02408094,
        -0.09192136, -0.03148066,  0.00213479, -0.08272441,  0.0209479 ,
         0.10059417, -0.10420871, -0.08304581, -0.0546338 ,  0.11259028,
       

In [255]:
'''
X shape: (5000, 401)
y shape: (5000, 1)
Theta1 shape: (25, 401)
Theta2 shape: (10, 26)
D1 shape: (25, 401)
D2 shape: (10, 26)
'''

'\nX shape: (5000, 401)\ny shape: (5000, 1)\nTheta1 shape: (25, 401)\nTheta2 shape: (10, 26)\nD1 shape: (25, 401)\nD2 shape: (10, 26)\n'

In [256]:
m, n =X.shape

In [257]:
def ForwardPropagation(X, theta1, theta2):
    z2 = theta1 @ X.T # z2 shape: (25, 1)
    a2 = expit(z2)
    a2 = np.insert(a2, 0, 1, axis = 0) # a2 shape: (26, 1)
    z3 = theta2 @ a2 # z3 shape: (10, 1)
    a3 = expit(z3) # a3 shape: (10, 1)
    return z2, a2, z3, a3
'''
def ForwardPropagation(X, T):
    z2 = T[0] @ X.T # z2 shape: (25, 1)
    a2 = expit(z2)
    a2 = np.insert(a2, 0, 1, axis = 0) # a2 shape: (26, 1)
    z3 = T[1] @ a2 # z3 shape: (10, 1)
    a3 = expit(z3) # a3 shape: (10, 1)
    return [(z2, a2), (z3, a3)]'''

'\ndef ForwardPropagation(X, T):\n    z2 = T[0] @ X.T # z2 shape: (25, 1)\n    a2 = expit(z2)\n    a2 = np.insert(a2, 0, 1, axis = 0) # a2 shape: (26, 1)\n    z3 = T[1] @ a2 # z3 shape: (10, 1)\n    a3 = expit(z3) # a3 shape: (10, 1)\n    return [(z2, a2), (z3, a3)]'

In [258]:
row1 = X[500].reshape((1, 401))
z2, a2, z3, a3 = ForwardPropagation(row1, Theta1, Theta2)

In [259]:
a3

array([[9.81212670e-01],
       [1.77130603e-02],
       [1.98089679e-03],
       [1.63701228e-03],
       [1.66140183e-03],
       [7.79555251e-04],
       [1.70240546e-03],
       [1.07248964e-02],
       [1.25880085e-04],
       [3.00769987e-04]])

In [260]:
print(f'z2 shape: {z2.shape}')
print(f'a2 shape {a2.shape}')
print(f'z3 shape {z3.shape}')
print(f'a3 shape {a3.shape}')

z2 shape: (25, 1)
a2 shape (26, 1)
z3 shape (10, 1)
a3 shape (10, 1)


In [261]:
def BackPropagation(X, y, theta1, theta2, lam = 0):
    '''
    y shape: (5000, 10)
    '''
    D2 = np.zeros((10, 26))
    D1 = np.zeros((25, 401))
    m, n = X.shape
    
    for i in range(m):
        rowi = X[i].reshape((1, n)) # rowi shape (1, 401)
        z2, a2, z3, a3 = ForwardPropagation(rowi, theta1, theta2)
        yi = y[i].reshape((10, 1))
        e3 = a3 - yi # e3 shape: (10, 1)
        # theta2 shape: (26, 10)
        # z2 shape: (25, 5000)
        e2 = theta2.T[1:, :].dot(e3) * SigmoidGradient(z2) # e2 shape: (25, 1)
        a2 = np.insert(a2[1:, :], 0, 1, axis = 0) # a2.shape (26, 1)
        D2 += e3 @ a2.T # D2 shape: (10, 26)
        D1 += e2 @ rowi #D1 shape: (25, 401)
        
    D1 = D1/float(m)
    D2 = D2/float(m)

    D2[:, 1:] = D2[:, 1:] + lam/m*theta2[:, 1:]
    D1[:, 1:] = D1[:, 1:] + lam/m*theta1[:, 1:]
    return D1, D2

In [262]:
# D1, D2 = BackPropagation(X, y_true, Theta1, Theta2)

In [263]:
D1, D2 = BackPropagation(X, y_true, t1_ini, t2_ini)

In [264]:
def GradientChecking(X, y, t1, t2):
    e = 10**(-4)
    m,n = t1.shape
    GradientApprox = np.zeros(m*n)
    for i in range(m*n):
        t_plus = t1.flatten()
        t_plus[i] = t_plus[i] + e
        t_plus = t_plus.reshape((m,n))
        t_minus = t1.flatten()
        t_minus[i] = t_minus[i] - e
        t_minus = t_minus.reshape((m,n))
        GradientApprox[i] = (nnCostFunction(X,y, t_plus, t2) - nnCostFunction(X,y, t_minus, t2))/(2*e)
    GradientApprox = GradientApprox.reshape((m,n))
    return GradientApprox


In [265]:
# D1_GradientApprox = GradientChecking(X, y, Theta1, Theta2)

In [266]:
# GradientTest = np.round(D1_GradientApprox[0], 4) == np.round(D1[0], 4)
# GradientTest[GradientTest == False]

# array([], dtype=bool)

In [267]:
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'Theta1 shape: {Theta1.shape}')
print(f'Theta2 shape: {Theta2.shape}')
print(f'D1 shape: {D1.shape}')
print(f'D2 shape: {D2.shape}')

X shape: (5000, 401)
y shape: (5000, 1)
Theta1 shape: (25, 401)
Theta2 shape: (10, 26)
D1 shape: (25, 401)
D2 shape: (10, 26)


In [268]:
# alpha = 0.1, iteration = 100
alpha = 0.01
iteration = 100
t1_ini, t2_ini, e_ini = randInitializeWeights(X, y_true, 25)

In [269]:
print(t1_ini.shape, t2_ini.shape)

(25, 401) (10, 26)


In [270]:
def NN(X, y, t1_ini, t2_ini, alpha, iter, lam = 0):
    m, n = X.shape
    K = 10
    y_true = np.zeros((m, K))
    t1 = t1_ini
    t2 = t2_ini
    Loss = []
    for i in range(m):
        y_true[i, y[i] - 1] = 1
    for i in range(iter):
        l = nnCostFunction(X, y, t1, t2, lam)
        if i % 100 == 0:
            print(l)
        Loss.append(l)
        d1, d2 = BackPropagation(X, y_true, t1, t2,lam)
        t1 -= alpha*d1
        t2 -= alpha*d2
        
    return t1, t2, Loss



        

In [ ]:
t1, t2, Loss = NN(X, y, t1_ini, t2_ini, alpha = 0.1, iter = 2000, lam =0)

1.2327384262832224
1.1576438878890807
1.0932710411736535
1.0373058824090322
0.9880762560151792
0.9443757065396254
0.9053278974447643
0.8702811357169158
0.838729123549602
0.8102564758614162


In [280]:
i = 3000
z2, a2, z3, a3 = ForwardPropagation(X[i], t1, t2)
prediction = np.argmax(a3) + 1
print(f'Prediction: {prediction}, Y: {y[i]}')

Prediction: 6, Y: [6]
